In [1]:
import pandas as pd
import numpy as np
import os
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer.autoguide import AutoDiagonalNormal
import seaborn as sns
import matplotlib.pyplot as plt
import math



smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.9.0')

We start with a simple model: choose a coin. Ground truth is the probability of heads is drawn from a beta(10,10). Then we do variational inference on this probability f by setting the variational family beta.

In [3]:
def model(data):
    # define the hyperparameters that control the Beta prior

    f1 = pyro.sample("latent_fairness_1", dist.Beta(10, 10))
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the Bernoulli
        # likelihood Bernoulli(f)
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f1), obs=data[i])

def guide_custome(model):

    # register the two variational parameters with Pyro.
    alpha_q1 = pyro.param("alpha_q1", torch.tensor(15.0),
                         constraint=dist.constraints.positive)
    beta_q1 = pyro.param("beta_q1", torch.tensor(15.0),
                        constraint=dist.constraints.positive)

    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness_1", dist.Beta(alpha_q1, beta_q1))


# setup the optimizer
adam_params = {"lr": 0.0005}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide_custome, optimizer, loss=Trace_ELBO())

data = []
for _ in range(6):
    data.append(torch.tensor(1.0))
for _ in range(4):
    data.append(torch.tensor(0.0))


# do gradient steps
n_steps = 2000
losses = []

for step in range(n_steps):
    loss = svi.step(data)
    losses.append(loss)
    if step % 100 == 0:
        print(f"Step {step}: Loss = {loss}")


alpha_q1 = pyro.param("alpha_q1").item()
beta_q1 = pyro.param("beta_q1").item()

inferred_mean = alpha_q1 / (alpha_q1 + beta_q1)
# compute inferred standard deviation
factor = beta_q1 / (alpha_q1 * (1.0 + alpha_q1 + beta_q1))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nBased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

Step 0: Loss = 7.201736569404602
Step 100: Loss = 7.047066807746887
Step 200: Loss = 6.743213534355164
Step 300: Loss = 6.695989549160004
Step 400: Loss = 6.914304554462433
Step 500: Loss = 7.117557883262634
Step 600: Loss = 7.098050117492676
Step 700: Loss = 7.06716001033783
Step 800: Loss = 7.061915338039398
Step 900: Loss = 7.093232035636902
Step 1000: Loss = 7.067635655403137
Step 1100: Loss = 7.069211959838867
Step 1200: Loss = 7.0644203424453735
Step 1300: Loss = 7.067325830459595
Step 1400: Loss = 7.069499492645264
Step 1500: Loss = 7.055732905864716
Step 1600: Loss = 7.06875467300415
Step 1700: Loss = 7.07213032245636
Step 1800: Loss = 7.053745985031128
Step 1900: Loss = 7.069560766220093

Based on the data and our prior belief, the fairness of the coin is 0.533 +- 0.090


Now we use the autoguide function autoDiagonalNormal to see if ADVI can solve the problem.

In [4]:
auto_guide = AutoDiagonalNormal(model)
svi = SVI(model, auto_guide, optimizer, loss=Trace_ELBO())

n_steps = 2000
losses = []

for step in range(n_steps):
    loss = svi.step(data)
    losses.append(loss)
    if step % 100 == 0:
        print(f"Step {step}: Loss = {loss}")

for name, value in pyro.get_param_store().items():
    print(f"{name} = {value.detach().cpu().numpy()}")


Step 0: Loss = 8.337636351585388
Step 100: Loss = 8.22623610496521
Step 200: Loss = 8.157578349113464
Step 300: Loss = 7.549182653427124
Step 400: Loss = 6.317932724952698
Step 500: Loss = 5.927767276763916
Step 600: Loss = 7.632397770881653
Step 700: Loss = 7.696081638336182
Step 800: Loss = 8.006523251533508
Step 900: Loss = 7.467472791671753
Step 1000: Loss = 7.819679260253906
Step 1100: Loss = 7.762797951698303
Step 1200: Loss = 7.794039845466614
Step 1300: Loss = 7.716073155403137
Step 1400: Loss = 5.458385944366455
Step 1500: Loss = 7.598601698875427
Step 1600: Loss = 7.570593237876892
Step 1700: Loss = 5.987421095371246
Step 1800: Loss = 7.318520188331604
Step 1900: Loss = 6.525657296180725
alpha_q1 = 16.006502151489258
beta_q1 = 14.038750648498535
AutoDiagonalNormal.loc = [0.13742647]
AutoDiagonalNormal.scale = [0.228038]


Try the mannual implementation.

In [ ]:
def guide_manual(model):
    

We see the inferred loc and scale is far from true. What's worse is that if we change the model to include two parameters. 

In [5]:
def model(data):
    # define the hyperparameters that control the Beta prior

    # sample f from the Beta prior
    f1 = pyro.sample("latent_fairness_1", dist.Beta(10, 10))
    f2 = pyro.sample("latent_fairness_2", dist.Beta(15, 10))
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the Bernoulli
        # likelihood Bernoulli(f)
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f1*f2), obs=data[i])

pyro.render_model(model, model_args=(data), render_distributions=True, render_params=True)


AssertionError: 

In [23]:


def guide_custome(model):

    # register the two variational parameters with Pyro.
    alpha_q1 = pyro.param("alpha_q1", torch.tensor(15.0),
                         constraint=dist.constraints.positive)
    beta_q1 = pyro.param("beta_q1", torch.tensor(15.0),
                        constraint=dist.constraints.positive)
    alpha_q2 = pyro.param("alpha_q2", torch.tensor(15.0),
                         constraint=dist.constraints.positive)
    beta_q2 = pyro.param("beta_q2", torch.tensor(15.0),
                        constraint=dist.constraints.positive)
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness_1", dist.Beta(alpha_q1, beta_q1))
    pyro.sample("latent_fairness_2", dist.Beta(alpha_q2, beta_q2))

# setup the optimizer
adam_params = {"lr": 0.0005}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide_custome, optimizer, loss=Trace_ELBO())

data = []
for _ in range(6):
    data.append(torch.tensor(1.0))
for _ in range(4):
    data.append(torch.tensor(0.0))


# do gradient steps
n_steps = 2000
losses = []

for step in range(n_steps):
    loss = svi.step(data)
    losses.append(loss)
    if step % 100 == 0:
        print(f"Step {step}: Loss = {loss}")


alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nBased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

Step 0: Loss = 8.728413105010986
Step 100: Loss = 8.319986701011658
Step 200: Loss = 9.983762741088867
Step 300: Loss = 9.776553630828857
Step 400: Loss = 5.533115863800049
Step 500: Loss = 8.818577289581299
Step 600: Loss = 9.619333267211914
Step 700: Loss = 8.925556898117065
Step 800: Loss = 8.452331185340881
Step 900: Loss = 7.522561430931091
Step 1000: Loss = 8.288254022598267
Step 1100: Loss = 8.806099653244019
Step 1200: Loss = 8.526350259780884
Step 1300: Loss = 8.105169773101807
Step 1400: Loss = 8.950329542160034
Step 1500: Loss = 8.330858945846558
Step 1600: Loss = 8.433443069458008
Step 1700: Loss = 8.39637565612793
Step 1800: Loss = 8.413408160209656
Step 1900: Loss = 8.325928211212158

Based on the data and our prior belief, the fairness of the coin is 0.530 +- 0.090


RuntimeError: shape '[]' is invalid for input of size 0
                  Trace Shapes:    
                   Param Sites:    
         AutoDiagonalNormal.loc 1  
       AutoDiagonalNormal.scale 1  
                  Sample Sites:    
_AutoDiagonalNormal_latent dist | 1
                          value | 1
         latent_fairness_1 dist |  
                          value |  

In [25]:
for name, value in pyro.get_param_store().items():
    print(f"{name} = {value.detach().cpu().numpy()}")

alpha_q = 15.89972972869873
beta_q = 14.127716064453125
AutoDiagonalNormal.loc = [0.14392696]
AutoDiagonalNormal.scale = [0.36030772]
alpha_q1 = 17.080364227294922
beta_q1 = 12.374051094055176
alpha_q2 = 20.10841178894043
beta_q2 = 10.899086952209473
